In [2]:
def scrape_tesla_locations():
    # Set up the Chrome WebDriver
    options = webdriver.ChromeOptions()
    options.add_experimental_option('excludeSwitches', ['enable-automation'])
    driver = webdriver.Chrome(options=options)
    
    # Open the main page
    driver.get('https://www.tesla.com/findus/list')
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'h1')))
        print("Page loaded successfully.")
    except TimeoutError:
        print("Page load timed out.")

    # Find all the locations' links
    all_links = driver.find_elements(By.XPATH, '//a')
    base_url = 'https://www.tesla.com/findus/list/'
    desired_links = []
    for link in all_links:
        if base_url in link.get_attribute('href'):
            desired_links.append(link.get_attribute('href'))

    driver.quit()

    return desired_links

In [1]:
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

In [4]:
list_link = scrape_tesla_locations()

Page loaded successfully.


In [5]:
len(list_link)

298

In [9]:
def scrape_tesla_infos(driver):
    # Wait for the page to load completely
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'h1')))
        print("Page loaded successfully.")
    except TimeoutError:
        print("Page load timed out.")

    # Extract title
    try:
        # Extract title
        titles = driver.find_element_by_xpath('/html/body/section/div/div/header/h1')
    except NoSuchElementException:
        print("Title element not found. Skipping this section.")
        return []


    # Find all links and corresponding addresses
    locations = driver.find_elements(By.CSS_SELECTOR, 'a[href*="/findus/location"]')
    addresses = driver.find_elements(By.CSS_SELECTOR, '.street-address-states')
    postals = driver.find_elements(By.CSS_SELECTOR, '.locality-city-postal')

    # Store the extracted information in a list of dictionaries
    dict = [{"title": titles.text}]

    for location, address, postal in zip(locations, addresses, postals):
        info = {
            "location": location.text,
            "url": location.get_attribute('href'),
            "address": address.text,
            "postal": postal.text
        }
        dict.append(info)

    return dict

In [10]:
def scrape_and_integrate(data_lists, links):
    desired_links = links
    all_data = data_lists

    # Set up the Chrome WebDriver
    options = webdriver.ChromeOptions()
    options.add_experimental_option('excludeSwitches', ['enable-automation'])
    driver = webdriver.Chrome(options=options)

    for link in desired_links:
        driver.get(link)
        single_web = scrape_tesla_infos(driver)
        all_data.extend(single_web)

    driver.quit()
    return all_data

In [11]:
data_to_write = scrape_and_integrate([], list_link[:100])

Page loaded successfully.


AttributeError: 'str' object has no attribute 'text'